# Chapter 13 - 특잇값 분해: 고윳값 분해의 다음 단계

In [1]:
import numpy as np
import pandas as pd
import sympy as sym
import scipy

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("ticks")
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg') # display figures in vector format
plt.rcParams.update({'font.size':14}) # set global font size

import statsmodels.api as sm


# 폰트 설정
%config InlineBackend.figure_format = 'retina'
plt.rcParams["axes.unicode_minus"] = False
plt.rc('font', family='NanumBarunGothic')

## 13-1. 대칭행렬의 특이 벡터와 고유 벡터는 같을까?
- 파이썬으로 무작위 $5*5 \quad A^{T}A$ 행렬을 사용해서 답해보자
- 그리고 덧셈 기법으로 대칭 행렬 $(A^{T}+A)$를 만들어 다시 시도해보자
- $A^{T}+A$의 부호에 주의하자

In [2]:
def compare_vt_v(matrix):
    # SVD 계산
    U, s, Vt = np.linalg.svd(matrix)

    # EVD 계산
    L, V = np.linalg.eig(matrix)

    # 특이 벡터와 고유벡터 같은지 확인
    print("Vt == V.T?", np.allclose(np.round(Vt, 3), np.round(V.T, 3)))
    print("")
    print("Vt: \n", np.round(Vt, 3))
    print("V.T: \n", np.round(V.T, 3))

In [3]:
# 무작위 행렬 생성
A = np.random.randn(5, 5)

In [4]:
compare_vt_v(A.T @ A)

Vt == V.T? False

Vt: 
 [[-0.363  0.799  0.291 -0.305  0.229]
 [-0.816 -0.392 -0.068  0.167  0.385]
 [-0.277  0.396 -0.54   0.485 -0.49 ]
 [ 0.064  0.017 -0.774 -0.571  0.266]
 [ 0.349  0.225 -0.142  0.564  0.7  ]]
V.T: 
 [[ 0.363 -0.799 -0.291  0.305 -0.229]
 [-0.816 -0.392 -0.068  0.167  0.385]
 [-0.277  0.396 -0.54   0.485 -0.49 ]
 [-0.349 -0.225  0.142 -0.564 -0.7  ]
 [-0.064 -0.017  0.774  0.571 -0.266]]


In [5]:
compare_vt_v(A.T + A)

Vt == V.T? False

Vt: 
 [[-0.059  0.945 -0.077  0.281 -0.136]
 [ 0.962 -0.032 -0.088  0.242 -0.084]
 [-0.109 -0.106  0.113  0.698  0.69 ]
 [ 0.212  0.249  0.774 -0.408  0.358]
 [ 0.118  0.181 -0.612 -0.457  0.608]]
V.T: 
 [[ 0.962 -0.032 -0.088  0.242 -0.084]
 [ 0.059 -0.945  0.077 -0.281  0.136]
 [-0.212 -0.249 -0.774  0.408 -0.358]
 [-0.109 -0.106  0.113  0.698  0.69 ]
 [ 0.118  0.181 -0.612 -0.457  0.608]]


## 13-2. 선택적으로 경제형 SVD 반환

In [6]:
# np.linalg.svd 공식문서 참고
# https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html
rng = np.random.default_rng()
a = rng.normal(size=(9, 6)) + 1j*rng.normal(size=(9, 6))
b = rng.normal(size=(2, 7, 8, 3)) + 1j*rng.normal(size=(2, 7, 8, 3))

In [7]:
a.shape

(9, 6)

In [8]:
# full_matrices=False로 했을 때는 U가 정방행렬이 아님
U1, S1, Vh1 = np.linalg.svd(a, full_matrices=False)
U1.shape, S1.shape, Vh1.shape

((9, 6), (6,), (6, 6))

In [9]:
np.allclose(a, np.dot(U1 * S1, Vh1))

True

In [10]:
# 특잇값 행렬 = np.diag(S1) (정방행렬)
smat = np.diag(S1)

In [11]:
np.allclose(a, np.dot(U1, np.dot(smat, Vh1)))

True

In [12]:
# full_matrices=True로 했을 때는 U 정방행렬로 만들어진다
U2, S2, Vh2 = np.linalg.svd(a, full_matrices=True)
U2.shape, S2.shape, Vh2.shape

((9, 9), (6,), (6, 6))

In [13]:
np.allclose(a, np.dot(U2[:, :6] * S2, Vh2))

True

In [14]:
# 특잇값 행렬 만드는 방식이 다름 (특잇값행렬이 정방행렬이 아니다)
smat2 = np.zeros(np.shape(a))
np.fill_diagonal(smat2, S2)

In [15]:
np.allclose(a, np.dot(U2, np.dot(smat2, Vh2)))

True

In [16]:
print(smat.shape)
print(smat2.shape)

(6, 6)
(9, 6)


## 13-7. 높은 최대계수 행렬의 명시적 왼쪽 역행렬 $((A^TA)^{-1}A^T)$과 $A$의 의사 역행렬을 계산
- MP 의사 역행렬이 최대 열계수 행렬의 왼쪽 역행렬과 같다는 것을 증명
- 넓은 최대계수 행렬의 오른쪽 역에 대해서도 증명

In [17]:
# 높은 최대계수 행렬 (m * n, m > n)
tall_matrix = rng.normal(size=(5, 3))

# 왼쪽 역행렬 구한다
Linv_tall = (np.linalg.inv(tall_matrix.T @ tall_matrix) @ tall_matrix.T)

In [18]:
Linv_tall.shape

(3, 5)

In [19]:
np.allclose(np.linalg.pinv(tall_matrix), Linv_tall)

True

In [20]:
# 넓은 최대계수 행렬 (m * n, m < n)
wide_matrix = rng.normal(size=(3, 5))

# 오른쪽 역행렬 구한다
Rinv_wide = (wide_matrix.T @ np.linalg.inv(wide_matrix @ wide_matrix.T))
Rinv_wide.shape

(5, 3)

In [21]:
# (Linv_wide @ wide_matrix).shape
np.allclose(wide_matrix @ Rinv_wide, np.eye(3))

True